In [123]:
from BMI_HBVmountain_Python import *
from Calibration_BRC import *
import os
import netCDF4 as nc
import emcee

In [124]:
forcing = pd.read_csv('Data/ThunderCreek/forcing_thundercreek.csv', index_col=[0], parse_dates=True)
pd.to_datetime(forcing.index);
forcing = forcing.reset_index(level=0)
for i in range(len(forcing)):
    forcing['time'][i] = forcing['time'][i].date()
forcing.set_index('time', inplace=True)

prec = forcing.prec_era5
temp = forcing.temp_era5
streamflow = forcing.streamflow

C:\Users\Vincent Hoogelander\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [125]:
def model(theta, prec, temp):
    [Tthresh, Fmelt, Mm, Pp, Kfast, Kslow, Fevap, Su_bare, Beta_bare, Imax_forest, Su_forest, Beta_forest,
     Imax_grass, Su_grass, Beta_grass, Imax_rip, Su_rip, Beta_rip, Kfast_rip, Prip] = theta
    
    bare = Parameters(Beta_bare, Fevap, 0.0, 0.0, Kfast, Fmelt, Mm, Pp, Su_bare, Tthresh)
    forest = Parameters(Beta_forest, Fevap, 0.0, Imax_forest, Kfast, Fmelt, Mm, Pp, Su_forest, Tthresh)
    grass = Parameters(Beta_grass, Fevap, 0.0, Imax_grass, Kfast, Fmelt, Mm, Pp, Su_grass, Tthresh)
    rip = Parameters(Beta_rip, Fevap, 0.0, Imax_rip, Kfast_rip, Fmelt, Mm, Pp, Su_rip, Tthresh)
    slow = Slow_Paramters(Kslow, Pp)
    
    param_list = [[bare, forest, grass, rip, slow]]
    model = climate_simulations(prec, temp, param_list)
    return model

In [126]:
def lnlike(theta,x, y, yerr):
    LnLike = -0.5 * np.sum(((y - model(theta, prec, temp)) / yerr)**2)#implement formula above
    return LnLike

In [127]:
def lnprior(theta):
    [Tthresh, Fmelt, Mm, Pp, Kfast, Kslow, Fevap, Su_bare, Beta_bare, Imax_forest, Su_forest, Beta_forest,
     Imax_grass, Su_grass, Beta_grass, Imax_rip, Su_rip, Beta_rip, Kfast_rip, Prip] = theta
    if (-2.0 < Tthresh < 2.0 and 1.0 < Fmelt < 5.0 and 0.001 < Mm < 1.0 and 0.1 < Pp < 0.9 and 0.1 < Kfast < 3.0 
    and 0.001 < Kslow < 0.1 and 0.4 < Fevap < 0.8 and 1.0 < Su_bare < 75 and 0.1 < Beta_bare < 2.0 and 1.0 < Imax_forest < 10.0 and
    50.0 < Su_forest < 750.0 and 0.1 < Beta_forest < 2.0 and 0.1 < Imax_grass < 7.0 and 5.0 < Su_grass < 400.0 and 0.1 < Beta_grass < 2.0 and 
    0.1 < Imax_rip < 8.0 and 5.0 < Su_rip < 400.0 and 0.1 < Beta_rip < 2.0 and 0.1 < Kfast_rip < 3.0 and 0.05 < Prip < 0.5):
        return 0.0
    else:
        return -np.inf

In [128]:
def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):#check if lp is infinite:
        return -np.inf
    return lp + lnlike(theta, x, y, yerr) #recall if lp not -inf, its 0, so this just returns likelihood

In [131]:
Streamflowerr = 0.05* np.mean(streamflow)
data = (forcing.index, streamflow ,Streamflowerr)
nwalkers = 51
niter = 5
initial = np.array([1.8, 2.73, 0.66, 0.33, 0.66, 0.03, 0.55, 12.3, 1.8, 9, 177.0, 1.1,
     3.6, 158.0, 0.75, 6.6, 7.3, 0.11, 2.2, 0.24])
ndim = len(initial)
p0 = [np.array(initial) + 1e-7 * np.random.randn(ndim) for i in range(nwalkers)]

In [132]:
def main(p0,nwalkers,niter,ndim,lnprob,data):
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=data, live_dangerously = True)

    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(p0, 100)
    sampler.reset()

    print("Running production...")
    pos, prob, state = sampler.run_mcmc(p0, niter)

    return sampler, pos, prob, state

In [133]:
sampler, pos, prob, state = main(p0,nwalkers,niter,ndim,lnprob,data)

Running burn-in...


TypeError: cannot convert the series to <class 'float'>

NameError: name 'sampler' is not defined

In [135]:
sampler

NameError: name 'sampler' is not defined